# Tutorial: Tabulated equation of state C code library

## Author: Leo Werneck

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#initialize_nrpy): Initialize NRPy+/Python modules
1. [Step 2](#identify_table_quantities): Identify EOS table quantities
1. [Step 3](#ccode_gen_helper_functions): C code generation helper functions
1. [Step 4](#interp_helpers): Interpolation helpers (<font color=orange>**in progress**</font>)
1. [Step 5](#general_wrappers): General wrapper functions for interpolation (<font color=orange>**in progress**</font>)
1. [Step 6](#nrpyeos_h): Generating the `NRPyEOS.h` header file (<font color=orange>**in progress**</font>)
1. [Step 7](#add_all_functions_to_the_dictionary): Add all functions to the dictionary (<font color=orange>**in progress**</font>)
1. [Step 8](#code_test): Code test - compiling and running a simple program (<font color=orange>**in progress**</font>)
1. [Step 9](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initialize_nrpy'></a>

# Step 1: Initialize NRPy+/Python modules \[Back to [Top](#toc)\]
$$\label{initialize_nrpy}$$

First load all necessary NRPy+/Python modules.

In [1]:
# Step 1: Initialize NRPy+/Python modules
import shutil, os, sys                   # Standard Python modules for multiplatform OS-level functions, benchmarking
from collections import namedtuple       # Standard Python: Enable namedtuple data type
sys.path.append(os.path.join("..","..")) # Add NRPy+'s base directory to Python's path
import outputC as outC                   # NRPy+: Core C code output module
import cmdline_helper as cmd             # NRPy+: Multi-platform Python command-line interface

<a id='identify_table_quantities'></a>

# Step 2: Identify EOS table quantities \[Back to [Top](#toc)\]
$$\label{identify_table_quantities}$$

We now identify the quantities from our EOS table. These will be used to:

1. Set the functions more easily;
1. Generate a C header file with $\mathtt{define}$'s which are compatible with this tutorial notebook;
1. Ensure that the interpolation happens in the same order the quantities appear in the table.

We do this by creating a [`namedtuple`](https://docs.python.org/3/library/collections.html#collections.namedtuple) that holds:

1. An integer that uniquely identifies the hydrodynamic quantity;
1. A description of the hydrodynamic quantity;
1. The "key", which will be the name used by C's $\mathtt{define}$;
1. The C variable name assumed by the functions.

In [2]:
# Step 2.a: Create the EOS named tuple
eos_tuple = namedtuple("eos_tuple","n desc key var")

# Step 2.b: Create tuples for all EOS quantities
P      = eos_tuple( 0, "Pressure"                          ,"NRPyEOS_press_key"  ,"P"       )
eps    = eos_tuple( 1, "Energy"                            ,"NRPyEOS_eps_key"    ,"eps"     )
S      = eos_tuple( 2, "Entropy"                           ,"NRPyEOS_entropy_key","S"       )
munu   = eos_tuple( 3, "Neutrino chemical potential"       ,"NRPyEOS_munu_key"   ,"munu"    )
cs2    = eos_tuple( 4, "Soundspeed"                        ,"NRPyEOS_cs2_key"    ,"cs2"     )
depsdT = eos_tuple( 5, "Derivative of eps w.r.t T"         ,"NRPyEOS_depsdT_key" ,"depsdT"  )
dPdrho = eos_tuple( 6, "Derivative of P w.r.t rho"         ,"NRPyEOS_dPdrho_key" ,"dPdrho"  )
dPdeps = eos_tuple( 7, "Derivative of P w.r.t eps"         ,"NRPyEOS_dPdeps_key" ,"dPdT"    )
muhat  = eos_tuple( 8, "mu_n - mu_p"                       ,"NRPyEOS_muhat_key"  ,"muhat"   )
mu_e   = eos_tuple( 9, "Electron chemical potential"       ,"NRPyEOS_mu_e_key"   ,"mu_e"    )
mu_p   = eos_tuple(10, "Proton chemical potential"         ,"NRPyEOS_mu_p_key"   ,"mu_p"    )
mu_n   = eos_tuple(11, "Neutron chemical potential"        ,"NRPyEOS_mu_n_key"   ,"mu_n"    )
X_a    = eos_tuple(12, "Alpha particle mass fraction"      ,"NRPyEOS_X_a_key"    ,"X_a"     )
X_h    = eos_tuple(13, "Heavy nuclei mass fraction"        ,"NRPyEOS_X_h_key"    ,"X_h"     )
X_n    = eos_tuple(14, "Neutron mass fraction"             ,"NRPyEOS_X_n_key"    ,"X_n"     )
X_p    = eos_tuple(15, "Proton mass fraction"              ,"NRPyEOS_X_p_key"    ,"X_p"     )
Abar   = eos_tuple(16, "Avg. mass number of heavy nuclei"  ,"NRPyEOS_Abar_key"   ,"Abar"    )
Zbar   = eos_tuple(17, "Avg. charge number of heavy nuclei","NRPyEOS_Zbar_key"   ,"Zbar"    )
Gamma  = eos_tuple(18, "Adiabatic index"                   ,"NRPyEOS_Gamma_key"  ,"Gamma"   )

<a id='ccode_gen_helper_functions'></a>

# Step 3: C code generation helper functions \[Back to [Top](#toc)\]
$$\label{ccode_gen_helper_functions}$$

We now write a series of helper functions that allow us to use lists of the `namedtuples` defined in the previous step to easily generate C functions to interpolate hydrodynamic quantities using the EOS table.

In [3]:
# Step 3: Helper functions
# Step 3.a: Set function name
def func_name(eos_params,auxvar_name):
    N_params = len(eos_params)
    name = "NRPyEOS"
    for i in range(N_params):
        if auxvar_name == "T" and i == len(eos_params)-1 and N_params > 1:
            name += "_and"
        name += "_"+eos_params[i].var.replace("_","")
    if auxvar_name != "T":
        name += "_and_T"
    name += "_from_rho_Ye_"+auxvar_name
    return name

# Step 3.b: Determine identation of function parameters
def param_indentation(c_type,name):
    indent = "  " # Parenthesis and space between type and name
    for i in range(len(c_type)):
        indent += " "
    for i in range(len(name)):
        indent += " "
    return indent

# Step 3.c: Set function parameters
def func_params(c_type,name,auxvar_name,eos_params):
    indent   = param_indentation(c_type,name)
    params   = "const NRPyEOS_params *restrict eos_params,\n"
    params  += indent+"const double rho,\n"
    params  += indent+"const double Ye,\n"
    params  += indent+"const double "+auxvar_name+",\n"
    N_params = len(eos_params)
    for i in range(N_params):
        if i == N_params-1:
            params += indent+"double *restrict "+eos_params[i].var
        else:
            params += indent+"double *restrict "+eos_params[i].var+",\n"
    return params

# Step 3.d: Set function body
def func_body(name,eos_params,auxvar):
    N_params   = len(eos_params)
    indent     = "  "
    body       = ""
    body      += indent+"// Step 1: Set EOS table keys\n"
    body      += indent+"const int keys["+str(N_params)+"] = {"
    for i in range(N_params):
        if i == N_params-1:
            body += eos_params[i].key
        else:
            body += eos_params[i].key+","
    body      += "};\n\n"
    body      += indent+"// Step 2: Declare EOS error report struct\n"
    body      += indent+"NRPyEOS_error_report report;\n\n"
    body      += indent+"// Step 3: Declare output array\n"
    body      += indent+"double outvars["+str(N_params)+"];\n\n"
    body      += indent+"// Step 4: Perform the interpolation\n"
    if auxvar == "T":
        body  += indent+"NRPyEOS_from_rho_Ye_T_interpolate_n_quantities( eos_params, "+str(N_params)+",rho,Ye,T, keys,outvars, &report );\n\n"
    else:
        body  += indent+"const root_finding_precision = 1e-10;"
        body  += """
  NRPyEOS_from_rho_Ye_aux_find_T_and_interpolate_n_quantities( eos_params, """+str(N_params)+""",root_finding_precision,
                                                               rho,Ye,"""+auxvar.var+","+auxvar.key+""", keys,outvars, T, &report );\n\n"""
    body      += indent+"// Step 5: Check for errors"
    body      += """
  if( report.error ) {
    fprintf(stderr,"(NRPyEOS) Inside """+name+""". Error message: %s (key = %d)",report.message,report.error_key);
  }\n\n"""
    body      += indent+"// Step 6: Update output variables\n"
    for i in range(N_params):
        body  += indent+"*"+eos_params[i].var+" = outvars["+str(i)+"];\n"
    return body

# Step 3.e: Functions for which the temperature is known
def Cfunc_known_T(eos_params_in):
    eos_params = sorted(eos_params_in)
    includes   = ["NRPyEOS.h"]
    desc       = "(c) 2022 Leo Werneck"
    c_type     = "void"
    name       = func_name(eos_params,"T")
    params     = func_params(c_type,name,"T",eos_params)
    body       = func_body(name,eos_params,"T")
    outC.add_to_Cfunction_dict(includes=includes,desc=desc,c_type=c_type,name=name,
                               params=params,body=body,enableCparameters=False)
#     print(outC.outCfunction("returnstring",
#                             includes=includes,desc=desc,c_type=c_type,name=name,
#                             params=params,body=body,enableCparameters=False))

# Step 3.f: Functinos for which the temperature is unknown
def Cfunc_unknown_T(auxvar,eos_params_in):
    eos_params = sorted(eos_params_in)
    includes   = ["NRPyEOS.h"]
    desc       = "(c) 2022 Leo Werneck"
    c_type     = "void"
    name       = func_name(eos_params,auxvar.var)
    params     = func_params(c_type,name,auxvar.var,eos_params)
    body       = func_body(name,eos_params,auxvar)
    outC.add_to_Cfunction_dict(includes=includes,desc=desc,c_type=c_type,name=name,
                               params=params,body=body,enableCparameters=False)
#     print(outC.outCfunction("returnstring",
#                             includes=includes,desc=desc,c_type=c_type,name=name,
#                             params=params,body=body,enableCparameters=False))

<a id='interp_helpers'></a>

# Step 4: Interpolation helpers \[Back to [Top](#toc)\]
$$\label{interp_helpers}$$

<a id='general_wrappers'></a>

# Step 5: General wrapper functions for interpolation \[Back to [Top](#toc)\]
$$\label{general_wrappers}$$

<a id='nrpyeos_h'></a>

# Step 6: Generating the `NRPyEOS.h` header file \[Back to [Top](#toc)\]
$$\label{nrpyeos_h}$$

<a id='add_all_functions_to_the_dictionary'></a>

# Step 7: Adding all functions to the dictionary \[Back to [Top](#toc)\]
$$\label{add_all_functions_to_the_dictionary}$$

The function below can be called to add all functions defined in this tutorial notebook to the C function dictionary.

In [4]:
# Step N: Add all C functions to the dictionary
def add_all_Cfuncs_to_dict():
    # Step N.a: Functions for which the temperature is known
    Cfunc_known_T([P])
    Cfunc_known_T([eps])
    Cfunc_known_T([P,eps])
    Cfunc_known_T([P,eps,S])
    Cfunc_known_T([P,eps,S,cs2])
    Cfunc_known_T([P,eps,depsdT])
    Cfunc_known_T([P,eps,mu_e,mu_p,mu_n,muhat])
    Cfunc_known_T([mu_e,mu_p,mu_n,muhat,X_p,X_n])
    
    # Step N.b: Functions for which the temperature is unknown
    # Step N.b.i: Temperature is determined using the specific internal energy
    Cfunc_unknown_T(eps,[P])
    Cfunc_unknown_T(eps,[P,S,depsdT])
    # Step N.b.ii: Temperature is determined using the pressure
    Cfunc_unknown_T(P  ,[eps,S])
    # Step N.b.iii: Temperature is determined using the entropy
    Cfunc_unknown_T(S  ,[P,eps])

<a id='code_test'></a>

# Step 8: Code test - compiling and running a simple program \[Back to [Top](#toc)\]
$$\label{code_test}$$

We now validate the implementation above by compiling a simple program and comparing the results against the trusted [`EOS_Omni`](https://einsteintoolkit.org/thornguide/EinsteinEOS/EOS_Omni/documentation.html) thorn from the [`Einstein Toolkit`](https://einsteintoolkit.org).

In [5]:
# Step N+1: Code test (in progress)
add_all_Cfuncs_to_dict()

<a id='latex_pdf_output'></a>

# Step 9: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Tabulated_Equation_of_State_Ccode_Library.pdf](Tutorial-Tabulated_Equation_of_State_Ccode_Library.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [6]:
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Tabulated_Equation_of_State_Ccode_Library")

Created Tutorial-Tabulated_Equation_of_State_Ccode_Library.tex, and
    compiled LaTeX file to PDF file Tutorial-
    Tabulated_Equation_of_State_Ccode_Library.pdf
